In [1]:
"""
Assignment 1 - main run method for mushroom dataset
"""
import sys
sys.path.append("C:\\users\\mccar\\miniconda3\\lib\\site-packages")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import validation_curve
from utils import learn_curve, val_curve



In [2]:
# get data
DATA_PATH = "data/mnist/mnist_train.csv"
train_df = pd.read_csv(DATA_PATH)
DATA_PATH = "data/mnist/mnist_test.csv"
test_df = pd.read_csv(DATA_PATH)
print(test_df.head())


   label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0      7    0    0    0    0    0    0    0    0    0  ...      0      0   
1      2    0    0    0    0    0    0    0    0    0  ...      0      0   
2      1    0    0    0    0    0    0    0    0    0  ...      0      0   
3      0    0    0    0    0    0    0    0    0    0  ...      0      0   
4      4    0    0    0    0    0    0    0    0    0  ...      0      0   

   28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  
0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0      0  

[5 rows x 785 columns]


In [3]:
# preprocessing
# none for now

In [4]:
# KNN
y_train = train_df["label"]
x_train = train_df.drop(["label"], axis=1)
y_test = test_df["label"]
x_test = test_df.drop(["label"], axis=1)

# Curve based on value of K - Validation curve
val_knn = False
if val_knn:
    k_accs = {}
    for k_val in range(1,11):
        knn = KNeighborsClassifier(n_neighbors=k_val)
        knn.fit(x_train, y_train)
        y_pred = knn.predict(x_test)
        k_accs[k_val] = accuracy_score(y_test, y_pred)
        print("K val: {}   acc: {}".format(k_val, k_accs[k_val]))
    val_curve(list(k_accs.keys()), list(k_accs.values()), "knn")

learn_knn = False
if learn_knn:
    # Shuffle the data without replacement - Learning curve
    x_accs = {}
    for x_perc in range(1,11):
        x_perc = x_perc / 10
        # shuffled_data = shuffle(x_train, random_state=RANDOM_SEED)
        x_percent_index = int(x_perc * len(x_train))
        # Take the first X% of the shuffled data
        first_x_percent = x_train[:x_percent_index]
        y_train_iter = y_train[:x_percent_index]
        knn = KNeighborsClassifier(n_neighbors=3)
        knn.fit(first_x_percent, y_train_iter)
        y_pred = knn.predict(x_test)
        x_accs[x_perc*10] = accuracy_score(y_test, y_pred)
        print("K val: {}   acc: {}".format(int(x_perc*10), x_accs[x_perc*10]))
    learn_curve(list(x_accs.keys()), list(x_accs.values()), "knn")


In [5]:
# Decision Tree
y_train = train_df["label"]
x_train = train_df.drop(["label"], axis=1)
y_test = test_df["label"]
x_test = test_df.drop(["label"], axis=1)

# DT - hyperparameter tuning - Validation curve
dt_max_depth = False

if dt_max_depth:
    # Max Depth
    # Define the range of values for the hyperparameter (e.g., max_depth)
    param_range = np.arange(1, 31)

    # Create a decision tree classifier
    dt_classifier = DecisionTreeClassifier(random_state=42)

    # Generate training and test scores using validation_curve
    train_scores, test_scores = validation_curve(
        dt_classifier, x_train, y_train, param_name="max_depth", param_range=param_range,
        cv=5, scoring="accuracy", n_jobs=-1
    )

    # Calculate mean and standard deviation for training and test scores
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(10, 6))
    plt.title("Validation Curve for Decision Tree - Max Depth")
    plt.xlabel("Max Depth")
    plt.ylabel("Accuracy")
    # Plot the mean training scores
    plt.plot(param_range, train_scores_mean, label="Training score", color="darkorange", marker='o')
    # Fill the area around the mean training scores
    plt.fill_between(param_range, train_scores_mean - train_scores_std,
                    train_scores_mean + train_scores_std, alpha=0.2, color="darkorange")
    # Plot the mean test scores
    plt.plot(param_range, test_scores_mean, label="Cross-validation score", color="navy", marker='o')
    # Fill the area around the mean test scores
    plt.fill_between(param_range, test_scores_mean - test_scores_std,
                    test_scores_mean + test_scores_std, alpha=0.2, color="navy")
    plt.legend(loc="best")
    plt.savefig("pngs/validation_curve_dt_max_depth.png", dpi=300)
    plt.close()





In [9]:
# Min samples to split
# Define the range of values for the hyperparameter (e.g., max_depth)
param_range = np.arange(1, 25)

# Create a decision tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42, max_depth=13)

# Generate training and test scores using validation_curve
train_scores, test_scores = validation_curve(
    dt_classifier, x_train, y_train, param_name="min_samples_split", param_range=param_range,
    cv=5, scoring="accuracy", n_jobs=-1
)

# Calculate mean and standard deviation for training and test scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.title("Validation Curve for Decision Tree - Max Depth")
plt.xlabel("Max Depth")
plt.ylabel("Accuracy")
# Plot the mean training scores
plt.plot(param_range, train_scores_mean, label="Training score", color="darkorange", marker='o')
# Fill the area around the mean training scores
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2, color="darkorange")
# Plot the mean test scores
plt.plot(param_range, test_scores_mean, label="Cross-validation score", color="navy", marker='o')
# Fill the area around the mean test scores
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2, color="navy")
plt.legend(loc="best")
plt.savefig("pngs/validation_curve_dt_min_samples_leaf.png", dpi=300)
plt.close()


In [ ]:

# DT - Amount of data - Learning curve

# DT - epochs - Learning curve